### 1. Import các thư viện cần thiết

In [1]:
import datetime
from pyspark.sql import SparkSession

### 2. Khởi tạo Spark

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("IE212_Lab2_Bai6") \
    .getOrCreate()
sc = spark.sparkContext

your 131072x1 screen size is bogus. expect trouble
25/11/25 21:56:48 WARN Utils: Your hostname, ELUCIDATOR resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/11/25 21:56:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/25 21:56:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/25 21:56:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/25 21:56:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/11/25 21:56:54 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/11/25 21:56:54 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/11/25 21:56:54 WARN Utils: Service 'SparkUI

### 3. Load dữ liệu

In [3]:
ratings1_path = "data/ratings_1.txt"
ratings2_path = "data/ratings_2.txt"

In [4]:
ratings_rdd = sc.textFile(f"{ratings1_path},{ratings2_path}")

### 4. Giải quyết bài toán

In [5]:
# 2. Xử lý Ratings: Trích xuất Năm từ Timestamp
# Schema: UserID, MovieID, Rating, Timestamp
def parse_rating_year(line):
    parts = line.split(",")
    rating = float(parts[2])
    timestamp = int(parts[3])
    
    # Chuyển đổi Unix Timestamp sang đối tượng datetime và lấy năm
    # Ví dụ: 978300760 -> 2000
    dt_object = datetime.datetime.fromtimestamp(timestamp)
    year = dt_object.year
    
    # Trả về Key là Năm để gom nhóm
    return (year, (rating, 1))

In [6]:
# 3. Tính toán thống kê theo Năm
# map: Chuyển dòng text thành (Year, (Rating, 1))
# reduceByKey: Cộng dồn điểm số và số lượt đánh giá cho từng năm
year_stats = ratings_rdd.map(parse_rating_year).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

In [7]:
# 4. Tính trung bình và Sắp xếp
# mapValues: Tính average = Sum / Count
# sortByKey: Sắp xếp kết quả theo năm tăng dần (để dễ nhìn xu hướng)
final_year_metrics = year_stats.mapValues(lambda x: (x[0] / x[1], x[1])).sortByKey()

In [8]:
# 5. Hiển thị kết quả
results = final_year_metrics.collect()

for year, (avg, total) in results:
    # Format output: Year - TotalRatings: xx, AverageRating: xx
    print(f"{year} - TotalRatings: {total}, AverageRating: {avg:.2f}")

2020 - TotalRatings: 184, AverageRating: 3.75
